Base script for comparison existance of output-format in the prompt

In [2]:
# review text

import sys
from pathlib import Path
import pandas as pd

sys.path.append('../../')

import _sample_reviews
import _prompts

from langchain_community.llms import Ollama

# YOUR REVIEW TEXT
review_text = _sample_reviews.sample_11

# YOUR OLLAMA MODEL OF THIS SCRIPT
llm = Ollama(model='gemma:2b-instruct-q4_0', temperature=0.4)

In [3]:
from datetime import datetime
from hashlib import sha224

# create an object with review_ID, review_text and the datetime of creation
def create_review(review_text, review_ID):    
    review_obj = {
        "review_ID": review_ID,
        "review_text": review_text,
        "datetime": datetime.now()
    }

    # create a hash of the review_text for creating unique collection with Chromadb
    hash = sha224(str(review_obj).encode()).hexdigest()
    review_obj['hash'] = hash

    return review_obj

# the temporary review_ID
review_ID = 123
# select a comment to test with
review_obj = create_review(review_text, review_ID)

print(review_obj)
print('\n\n')
print('Len of review_text:', len(review_text.split()))

{'review_ID': 123, 'review_text': 'overall good magic card game. the only issue i can really see is the random algorithem for shuffeling the deck, more times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size. i would also suggest to the devs to allow specific card buying for coins as its really hard to get a single card from an entire set of cards, and yes i would expect the cost to be much higher than for booster packs, adjusted for card value and rareity.', 'datetime': datetime.datetime(2024, 4, 4, 11, 18, 51, 480517), 'hash': 'eccd52c55e2e02a974233294710d851b465d77a0eee1ffaf33dac12b'}



Len of review_text: 94


In [4]:
# define callbacks for detecting token usage
# copied from ../previous_tests/ollama_get_token_usage_example.ipynb

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

In [5]:
import chromadb

# use docker
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [6]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma

# also change this to use Mistral Embeddings
embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

text_spliter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base', chunk_size=250, chunk_overlap=40
)

docs = text_spliter.create_documents(
    [review_text], metadatas=[{"source":"review_01"}]
)

temp_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_func,
    collection_name=review_obj['hash']
)

retriever = temp_db.as_retriever(search_kwargs={"k": 5})

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michaelcheng/miniforge3/envs/fyp-test-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


---

prompting

In [7]:
# global constants
GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']

3-3-4

with one-shot output format provided

In [8]:
# 3-3-4
# modified from main branch

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Gameplay": "The game is overall good, but the random algorithm for shuffling the deck can be frustrating. It can sometimes take a long time to get a major mana pocket or no mana at all for half the deck, regardless of the deck size.",
  "Narrative": "The game has a rich narrative that is well-written and engaging.",
  "Accessibility": "The game is accessible for players of all skill levels, but it can be challenging for beginners. The game has a lot of depth and complexity that can be overwhelming for newcomers."
}

{"Gameplay": "The game is overall good, but the random algorithm for shuffling the deck can be frustrating. It can sometimes take a long time to get a major mana pocket or no mana at all for half the deck, regardless of the deck size.",
"Narrative": "The game has a rich narrative that is well-written and engaging.",
"Accessibility": "The game is accessible for players of all skill levels, but it can be challenging for beginners. The game has a lot of depth and complexi

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Sound": "...",
  "Graphics & Art Design": "...",
  "Performance": "..."
}

NA
--------------------
{
  "Bug": "...",
  "Suggestion": "Improve random algorithm for shuffling the deck.",
  "Price": "...",
  "Overall": "Good"
}

NA
--------------------


In [9]:
token_usage_json_list

[{'token_usage': 484, 'time_costed': 2371282000},
 {'token_usage': 294, 'time_costed': 509249000},
 {'token_usage': 308, 'time_costed': 573917000}]

3-3-4 without output format provided

In [10]:
# without output_format

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        # "output_format": output_format,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Gameplay": "The gameplay is overall good, but the random algorithm for shuffling the deck can be frustrating.",
  "Narrative": "The narrative is not very detailed, but it is well-paced and engaging.",
  "Accessibility": "The game is accessible for players of all skill levels, but the random algorithm can make it difficult for new players to get started."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Sound": "NA",
  "Graphics & Art Design": "NA",
  "Performance": "NA"
}
--------------------
{
  "Bug": "Random shuffling algorithm for deck is inconsistent. Either you hit a major mana pocket or no mana at all for half the deck, regardless of deck size.",
  "Suggestion": "Allow specific card buying for coins. This would make it easier to get single cards from entire sets of cards.",
  "Price": "The price should be much higher than for booster packs, adjusted for card value and rarity.",
  "Overall": "overall good magic card game."
}
--------------------


In [11]:
token_usage_json_list

[{'token_usage': 283, 'time_costed': 900603000},
 {'token_usage': 235, 'time_costed': 433494000},
 {'token_usage': 308, 'time_costed': 1082335000}]

---

2-2-2-2-2 prompting

with output_format provided

In [12]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Gameplay": "...", "Narrative": "..."}

{"aspects": {
  "Gameplay": "the only issue i can really see is the random algorithem for shuffeling the deck, more times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size. i would also suggest to the devs to allow specific card buying for coins as its really hard to get a single card from an entire set of cards, and yes i would expect the cost to be much higher than for booster packs, adjusted for card value and rareity."
}}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Accessibility": "...", "Sound": "..."}

{"aspects": {
  "Accessibility": "The game is generally accessible for players of all skill levels.",
  "Sound": "The sound design is good and enhances the gameplay experience."
}}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Graphics & Art Design": "...",
  "Performance": "More times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size."
}

NA
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Bug": "...", "Suggestion": "Improve random shuffling algorithm for deck shuffeling"}

{"JSON": "{"Bug": "...", "Suggestion": "Improve random shuffling algorithm for deck shuffeling"}"}
--------------------
{"Price": "...", "Overall": "overall good magic card game"}

{"Price": "...", "Overall": "more times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size"}
--------------------


In [13]:
token_usage_json_list

[{'token_usage': 366, 'time_costed': 1261285000},
 {'token_usage': 297, 'time_costed': 649814000},
 {'token_usage': 301, 'time_costed': 603934000},
 {'token_usage': 287, 'time_costed': 557272000},
 {'token_usage': 296, 'time_costed': 641754000}]

2-2-2-2-2 wihtout output format

In [14]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


```json
{
  "Gameplay": "The game is overall good and engaging, but the random algorithm for shuffling the deck can be frustrating, especially when you're trying to get a major mana pocket or no mana at all for half of the deck. The cost of cards should also be adjusted to be more fair, as it can be difficult to get a single card from an entire set of cards."
}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**
```json
{
  "Accessibility": "NA",
  "Sound": "NA"
}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**
```json
{
  "Graphics & Art Design": "The game has beautiful and detailed graphics and art design.",
  "Performance": "The game runs smoothly and has a low latency."
}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


```json
{
  "Bug": "Random shuffling algorithm can be frustrating",
  "Suggestion": "Allow specific card buying for coins"
}
```
--------------------
**Aspect 1: Price**
Paragraph: The price of the game is not specified in the review, so I cannot extract the requested information from the context.

**Aspect 2: Overall**
Paragraph: The overall review is positive and suggests that the game is a good magic card game.
--------------------


In [15]:
token_usage_json_list

[{'token_usage': 286, 'time_costed': 934059000},
 {'token_usage': 227, 'time_costed': 411963000},
 {'token_usage': 250, 'time_costed': 590859000},
 {'token_usage': 233, 'time_costed': 463925000},
 {'token_usage': 260, 'time_costed': 707268000}]

---

5-5 with output format provided

In [16]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
"Gameplay": "...",
"Narrative": "...",
"Accessibility": "...",
"Sound": "...",
"Graphics & Art Design": "..."}

NA
--------------------
{
  "Performance": "...",
  "Bug": "Random algorithm for shuffling the deck",
  "Suggestion": "Allow specific card buying for coins",
  "Price": "Higher than booster packs",
  "Overall": "Good"
}

NA
--------------------


In [17]:
token_usage_json_list

[{'token_usage': 323, 'time_costed': 535480000},
 {'token_usage': 335, 'time_costed': 719717000}]

5-5 without output format provided

In [18]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Gameplay": "Overall good magic card game",
  "Narrative": "NA",
  "Accessibility": "NA",
  "Sound": "NA",
  "Graphics & Art Design": "NA"
}
--------------------
{
  "Performance": "The game runs smoothly and is enjoyable to play.",
  "Bug": "The random shuffling algorithm can be frustrating, as it can lead to situations where you either get a major mana pocket or no mana at all for half the deck.",
  "Suggestion": "Allow players to buy specific cards for coins, as it can be quite difficult to get a single card from an entire set of cards.",
  "Price": "The game is priced higher than other booster pack deals, but the quality of the cards is high and the gameplay is enjoyable.",
  "Overall": "A good magic card game with a few minor issues that could be addressed by the developers."
}
--------------------


In [19]:
token_usage_json_list

[{'token_usage': 261, 'time_costed': 609305000},
 {'token_usage': 353, 'time_costed': 1455237000}]

---

1 aspect per prompt, with output format

In [20]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


```json
{"Gameplay": "The game's random shuffling algorithm can be frustrating, as it often results in either a major mana pocket or no mana at all for half of the deck, regardless of the deck size. The cost of cards can also be very high, and it can be difficult to get a single card from an entire set of cards, especially for coins."}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Narrative": "..."}

{"Aspects": "Narrative"}
--------------------


...

{"Bug": "Random algorithm for shuffling the deck is too unpredictable and leads to inconsistent outcomes."}

{"JSON": "{"Bug": "Random algorithm for shuffling the deck is too unpredictable and leads to inconsistent outcomes."}}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Suggestion": "Improve the random algorithm for shuffling the deck to ensure more consistent mana distribution."
}

```
{"Suggestion": "Improve the random algorithm for shuffling the deck to ensure more consistent mana distribution."}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
"Price": "NA"
}
--------------------
{"Overall": "overall good magic card game"}

{"NA": "NA"}
--------------------


In [21]:
token_usage_json_list

[{'token_usage': 261, 'time_costed': 609305000},
 {'token_usage': 353, 'time_costed': 1455237000},
 {'token_usage': 313, 'time_costed': 893187000},
 {'token_usage': 246, 'time_costed': 294778000},
 {'token_usage': 287, 'time_costed': 658339000},
 {'token_usage': 285, 'time_costed': 640746000},
 {'token_usage': 258, 'time_costed': 321757000},
 {'token_usage': 280, 'time_costed': 593663000},
 {'token_usage': 279, 'time_costed': 586237000},
 {'token_usage': 283, 'time_costed': 622553000},
 {'token_usage': 244, 'time_costed': 275897000},
 {'token_usage': 251, 'time_costed': 337655000}]

1 aspect per prompt, without output format

In [22]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


```json
{
  "Gameplay": "The game's random shuffling algorithm can be frustrating, as players may find themselves with no mana or major mana pockets for half of the deck. Additionally, the cost of buying cards with coins can be quite high, especially for players who want to build a specific deck."
}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Narrative": "The game's narrative is well-written and engaging."
}

```
{
  "aspects": [
    {
      "name": "Narrative",
      "paragraph": "The game's narrative is well-written and engaging."
    }
  ]
}
```
--------------------


...

{
  "Bug": "Random algorithm for shuffling the deck makes it difficult to get major mana pockets or no mana at all for half the deck."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
  "Suggestion": "Improve random shuffling algorithm for deck shuffling."
}

```
{
  "Suggestion": "Improve random shuffling algorithm for deck shuffling.",
  "Suggestion": "Allow specific card buying for coins."
}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


```json
{
  "Price": "NA"
}
```
--------------------
{
  "Overall": "overall good magic card game."
}

```
{
  "Overall": "overall good magic card game."
}
```
--------------------
